# 240213 source_sequencing_analysis_v3

- Notebook to analyse sequencing data from Source Bioscience
- Create a copy of this notebook
- Add the FWD and REV .seq files to a folder on GDrive - keep these in the format supplied by Source Bioscience (e.g. '..._A01.seq')
- Fill out the variables in the cell below:
  - gene_seq: the nucleotide sequence of the reference gene from start ATG to STOP codon (5' to 3')
  - stop_downstream_15bp: 15 bp downstream of the STOP codon (5' to 3') - **not including the STOP codon**
  - fwd_seq_path: the GDrive path for the folder containing the FWD .seq files
  - rev_seq_path: the GDrive path for the folder containing the REV .seq files
- Once variables are filled out, press 'Runtime > Run all'
- Once the script has stopped running, inspect the alignments output at the bottom of the script

Notes:
*   Sequencing reads are not filtered for quality scores
*   Indices shown on alignments are not always accurate - shifts can occur from misalignments if the sequencing quality is poor





In [17]:
gene_seq = "ATGTCCAATTCATACCCGCAAAAACTGCAATTCCACATCGATGGTGAATGGATCGATGTTGGTGAGCGGGATGTTCATCATGTAGTGAACCCGGCGACCGGCAAGGTTATTTCAGATTTGCCTAAAGCAACCAGAGCTGATCTGGATCGTGCTCTGGATGCTGCGGACCGTGCCTTTCCGATTTGGCGTGACACGCCGGTGGGTGAACGCGCAGCGATCTTACATAAAGCCGCTGATCTTATGCGTGAACGGGCGCCTGAAATTGGGAGACTGATGACCGCAGAGCAAGGGAAACCATTACCCCAGGCGATTGGGGAAGTCACTGCATCAGCTTCACATTTTGACATCATGGCCGAAGAAGCCAAACGATGTTACGGCCGTGTATTGGTTCGCCCAGCAGGTCAGAACAGTTTCGTCAAATATCAGCCGGTTGGGCCAGTTGCGGCCTTTAGTCCATGGAATTTCCCTGTATTCACCCCAGTGCGCAAGCTTGCTCCTGCGATTGCTGCAGGGTGTTCGATAATTCTCAAACCTGCAGAGGAAACCCCGGCTAGTCCTATAGAGATGATACGCTGCCTGGTAGATGCGGGCCTGCCCGCCGGTGTTGCACAAGTTGTATTTGGCGATCCGGCAGAAGTCTCTAGCCATTTAATCGCCTCGCCGGTTATCCGCAAGATTAGCTTTACAGGTTCCGTCCCCGTCGGCAAACATCTGATGAAGTTGGCAGCAGAGGGTATGAAACGCACTACAATGGAGCTGGGTGGACACGCTCCTGTGCTGGTGTTCGATGACTGTGATCTGGAGAAAACGCTTGATCTTGTGGTGACAGGAAAATTTCGTAATGCAGGTCAGGTCTGCGTTTCTCCAACCAGATTCTATGTCCAGTCTGGCATCTATGACCGATTCGAAAAGGCCTTTACAGAGCGTGCAGCCAAGGTTGCGGTAGGTGATGGTTTTGCTGACGGAATTGAAATGGGTCCGCTGGCCAATCCCCGTCGACCCAGTTCTGTCGGCGATATGATCAGTGATGCCAAAAGCAAAGGCGCGGAAGTGATGCTGGGCGGGCACCGGCAGGATAAAGATGGCTTTTTCTTTGATCCGACGGTGTTATCTCATGTTCCTCTCGACGCAAATATAATGAACGATGAGCCGTTTGGACCGGTAGCGGTGATGCGGCCGTTTGACACGCTCGACGAAGCGATTGAACAGGCCAACCGCCTGCCCTTGGGATTAGCAGCTTATGCCTTTACTTCCTCACTTCACACAGCCAATATGGTTGGTGACAAAATTGAAGCTGGAATGGTGGCGATTAACAATCTCACTGTAAGCCCAGGCGACGCGCCATTTGGCGGTGTTAAGGAAAGCGGGCATGGATCGGAAGACGGACCTGATGGGTTAAAAGCCTATATGGTGACGAAAGCAATACACCAGGCTTAA" # @param {type:"string"}
stop_downstream_15bp = "CTCGAGCACCACCAC" # @param {type:"string"}
fwd_seq_path = "/content/drive/My Drive/Seq_data/LIB053_F" # @param {type:"string"}
rev_seq_path = "/content/drive/My Drive/Seq_data/Lib053_R" # @param {type:"string"}

In [18]:
from google.colab import drive # Importing drive from GDrive
drive.mount('/content/drive') # Mounting the drive
import glob #  Finds all the path names matching a specified pattern using python
from natsort import natsorted # The natsort() function sorts an array by using a "natural order" algorithm
! pip install biopython # Install BioPython
from Bio import pairwise2 # Import pairwise2 aligner
from Bio.Seq import Seq # Import Seq class
from Bio.pairwise2 import format_alignment

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Create a dictionary that links the well position, sequence direction and sequencing reads

source_seqs = {}

source_fastas_fwd = glob.glob(fwd_seq_path + '/*.seq')
source_fastas_rev = glob.glob(rev_seq_path + '/*.seq')

for fasta_file in source_fastas_fwd:
  seq_name = fasta_file.split('_')[-1].strip('.seq')
  source_fasta = open(fasta_file, 'r')
  for line in source_fasta:
    source_seqs[seq_name + '_fwd'] = line

for fasta_file in source_fastas_rev:
  seq_name = fasta_file.split('_')[-1].strip('.seq')
  source_fasta = open(fasta_file, 'r')
  for line in source_fasta:
    source_seqs[seq_name + '_rev'] = line

In [20]:
# Order the 'source_seqs' dictionary 'naturally' using natsorted()

ordered_source_seqs = {}

for i in natsorted(source_seqs):
  ordered_source_seqs[i] = source_seqs[i]

In [21]:
ordered_source_seqs

{'A01_fwd': 'NNNNNNNNANNNNNNNCCNNCNNNAATATTTTGTTTAACTTTAAGAAGGAGATATACATATGTCCAATTCATACCCGAAAAAACTGCAATTCCACATCGATGGTGAATGGATCGATGTTGGTGAGCGGGATGTTCATCATGTAGTGAACCCGGCGACCGGCAAGGTTATTTCAGATTTGCCTAAAGCAACCAGAGCTGATCTGGATCGTGCTCTGGATGCTGCGGACCGTGCCTTTCCGATTTGGCGTGACACGCCGGTGGGTGAACGCGCAGCGATCTTACATAAAGCCGCTGATCTTATGCGTGAACGGGCGCCTGAAATTGGGAGACTGATGACCGCAGAGCAAGGGAAACCATTACCCCAGGCGATTGGGGAAGTCACTGCATCAGCTTCACATTTTGACATCATGGCCGAAGAAGCCAAACGATGTTACGGCCGTGTATTGGTTCGCCCAGCAGGTCAGAACAGTTTCGTCAAATATCAGCCGGTTGGGCCAGTTGCGGCCTTTAGTCCATGGAATTTCCCTGTATTCACCCCAGTGCGCAAGCTTGCTCCTGCGATTGCTGCAGGGTGTTCGATAATTCTCAAACCTGCAGAGGAAACCCCGGCTAGTCCTATAGAGATGATACGCTGCCTGGTAGATGCGGGCCTGCCCGCCGGTGTTGCACAAGTTGTATTTGGCGATCCGGCAGAAGTCTCTAGCCATTNNATCGCCTCGCCGGTTATCCGCAAGATTAGCTTTACAGGTTCCGTCCCCGTCGCCAAACATCTGATGAAGTTGGCAGCAGAGNGTATGAAACNNNCTACATTGGAGCTGGGTGGACACGCTCCTGTGCTGGTGGTCGATGACTGTGATCTGGAGAAAACGCTTGATCTTGTTGTGACAGGNANTTTTCTNATGCAGGTAGGTCTTGGTTTCTCCAACNGGATTCTATGTCCAGGCTGGGCATNNATGACCCGATTCNNAAAANGCNNNTTACTGNGCG

In [22]:
# Stores various variables to trim sequences in the cell below

aa_cds = Seq(gene_seq).translate() # Translates the gene into an AA sequence

shine_dalgarno_seq = 'AAGGAGATATACAT' # Stores the Shine-Dalgarno sequence directly upstream of the start ATG

gene_end = gene_seq[len(gene_seq)-8:] # Stores the final 8 bp of the gene sequence

stop_downstream_15bp_rev_comp = Seq(stop_downstream_15bp).reverse_complement() # Stores the reverse complement of the 15 bp downstream of the STOP codon

In [23]:
# Creates a dictionary that stores the trimmed sequences

trimmed_source_seqs = {} # Creates an empty dictionary to store the trimmed sequences

for key in ordered_source_seqs:
  if 'fwd' in key:
    start = ordered_source_seqs[key].find(shine_dalgarno_seq)
    trimmed_seq = ordered_source_seqs[key][start+(len(shine_dalgarno_seq)):] # Trim the fwd sequencing reaction from the start ATG onwards
    trimmed_source_seqs[key] = trimmed_seq
  if 'rev' in key:
    rev_end = ordered_source_seqs[key].find(str(stop_downstream_15bp_rev_comp))
    ordered_source_seqs[key] = ordered_source_seqs[key][rev_end+len(stop_downstream_15bp):]
    if len(ordered_source_seqs[key]) % 3 == 2:
      ordered_source_seqs[key] = ordered_source_seqs[key][:len(ordered_source_seqs[key])-2]
    elif len(ordered_source_seqs[key]) % 3 == 1:
      ordered_source_seqs[key] = ordered_source_seqs[key][:len(ordered_source_seqs[key])-1]
    rev_comp = Seq(ordered_source_seqs[key]).reverse_complement()
    trimmed_source_seqs[key] = str(rev_comp[150:]) # Remove the first 150 bp from the rev_comp sequence as this is typically low quality

In [16]:
def seq_alignment(seq1, seq2):
  matches = (pairwise2.align.globalms(seq1, seq2, 2, -1, -100, -100)) # 2 pts for match, -1 for mismatch, -100 for gap introduction, -100 for gap extension
  print(format_alignment(*matches[0]))


for key in trimmed_source_seqs:
  seq = Seq(trimmed_source_seqs[key])
  if len(seq) % 3 == 1: # If one extra bp, add two extra to be a complete codon
    seq = seq + 'N' + 'N'
  elif len(seq) % 3 == 2: # If two extra bp, add one extra to be a complete codon
    seq = seq + 'N'
  seq_AA = seq.translate()
  if len(seq) > 100:
    print('1--------10--------20--------30--------40--------50--------60--------70--------80--------90--------100-------110-------120-------130-------140-------150-------160-------170-------180-------190-------200-------210-------220-------230-------240-------250-------260-------270-------280-------290-------300-------310-------320-------330-------340-------350-------360-------370-------380-------390-------400-------410-------420-------430-------440-------450-------460-------470-------480-------490-------500-------510-------520-------530-------540-------550-------560-------570-------580-------590-------600-------610-------620-------630-------640-------650-------660-------670-------680-------690-------700')
    seq_alignment(aa_cds, seq_AA)
    print(key,'\n')
  else:
    print(key + ' is less than 100 bp long','\n')

1--------10--------20--------30--------40--------50--------60--------70--------80--------90--------100-------110-------120-------130-------140-------150-------160-------170-------180-------190-------200-------210-------220-------230-------240-------250-------260-------270-------280-------290-------300-------310-------320-------330-------340-------350-------360-------370-------380-------390-------400-------410-------420-------430-------440-------450-------460-------470-------480-------490-------500-------510-------520-------530-------540-------550-------560-------570-------580-------590-------600-------610-------620-------630-------640-------650-------660-------670-------680-------690-------700
MSNSYPQKLQFHIDGEWIDVGERDVHHVVNPATGKVISDLPKATRADLDRALDAADRAFPIWRDTPVGERAAILHKAADLMRERAPEIGRLMTAEQGKPLPQAIGEVTASASHFDIMAEEAKRCYGRVLVRPAGQNSFVKYQPVGPVAAFSPWNFPVFTPVRKLAPAIAAGCSIILKPAEETPASPIEMIRCLVDAGLPAGVAQVVFGDPAEVSSHLIASPVIRKISFTGSVPVGKHLMKLAAEGMKRTTMELGGHAPVLVFDDCDLEKTLDLVVTGKFRNAGQVCVSPTRFYVQSG